# Main

### Imports

In [5]:
import numpy as np
from keras.models import load_model, Model

### Current values

In [38]:
# ceci est un test

actual_temp = np.ndarray(30, dtype = "float32")
actual_temp.fill(0.30) # (11.36, 40.0)

actual_co2 = np.ndarray(30, dtype = "float32")
actual_co2.fill(0.3) # (84, 1990)

actual_lum = np.ndarray(90, dtype = "float32")
actual_lum.fill(0.6) # (0, 1000)

actual_hum = np.ndarray(30, dtype = "float32")
actual_hum.fill(0.7) # (min, max) : (30,75)

actual_features = [actual_co2, actual_hum, actual_lum, actual_temp]

# actual_temp, actual_co2, actual_lum, actual_hum

In [24]:
model_name_1 = "models/model_ilot1.co2_10_1.h5"
model_name_2 = "models/model_ilot1.temp_10_1.h5"
model_name_3 = "models/model_ilot1.lum_30_1.h5"
model_name_4 = "models/model_ilot1.hum_10_1.h5"

In [39]:
model1 = load_model(model_name_1)
Y_test_pred1 = model1.predict(actual_co2.reshape(3,10,1))

model2 = load_model(model_name_2)
Y_test_pred2 = model2.predict(actual_temp.reshape(3,10,1))

model3 = load_model(model_name_3)
Y_test_pred3 = model3.predict(actual_lum.reshape(3,30,1))

model4 = load_model(model_name_4)
Y_test_pred4 = model4.predict(actual_hum.reshape(3,10,1))

In [30]:
Y_test_pred1, Y_test_pred2, Y_test_pred3, Y_test_pred4

(array([[0.08869418],
        [0.08869418],
        [0.08869418]], dtype=float32), array([[0.3507897],
        [0.3507897],
        [0.3507897]], dtype=float32), array([[0.7173173],
        [0.7173173],
        [0.7173173]], dtype=float32), array([[0.4577512],
        [0.4577512],
        [0.4577512]], dtype=float32))

# annotationUnique.py

In [26]:
# Annotation d'une valeur de température selon les seuils définis
def temperature(valeur):
    if(valeur < 23.2 and valeur > 20.8):
        return 5
    elif(valeur < 24.5 and valeur > 19.5):
        return 4
    elif(valeur < 26 and valeur > 18):
        return 3
    elif(valeur < 27 and valeur > 17):
        return 2
    elif(valeur < 28 and valeur > 16):
        return 1
    else:
        return 0

# Annotation d'une valeur de luminosité selon les seuils définis
def luminosite(valeur):
    if(valeur > 600 and valeur < 900):
        return 5
    elif(valeur > 550 and valeur < 950):
        return 4
    elif(valeur > 500 and valeur < 1000):
        return 3
    elif(valeur > 400 and valeur < 1100):
        return 2
    elif(valeur > 200 and valeur < 1300):
        return 1
    else:
        return 0

# Annotation d'une valeur de co2 selon les seuils définis
def co2(valeur):
    if(valeur<400):
        return 5
    elif(valeur<600):
        return 4
    elif(valeur<800):
        return 3
    elif(valeur<1000):
        return 2
    elif(valeur<1200):
        return 1
    else:
        return 0

# Annotation d'une valeur d'humidité selon les seuils définis
def humidite(valeur):
    if(valeur < 55 and valeur > 45):
        return 5
    elif(valeur < 58 and valeur > 42):
        return 4
    elif(valeur < 63 and valeur > 37):
        return 3
    elif(valeur < 70 and valeur > 30):
        return 2
    elif(valeur < 80 and valeur > 20):
        return 1
    else:
        return 0

# Annotation d'une situation globale pour un capteur, en regroupant les valeurs trouvées durant des périodes de deux minutes, pour les différentes mesures.
def annotationCapteur(valeurTemperature, valeurLuminosite, valeurCo2, valeurHumidite):
    annotationRetour = 0
    # Définition de poids, selon l'importance d'un critère pour l'humain
    # Poids définis ici après un vote entre les membres du groupe
    poidsTemperature = 0.3
    poidsLuminosite = 0.2
    poidsCo2 = 0.4
    poidsHumidite = 0.1
    annotationRetour += poidsTemperature * temperature(valeurTemperature)
    annotationRetour += poidsLuminosite * luminosite(valeurLuminosite)
    annotationRetour += poidsCo2 * co2(valeurCo2)
    annotationRetour += poidsHumidite * humidite(valeurHumidite)

    return str(annotationRetour);


# VALEUR DE CONFORT PREDICTED !!

In [40]:
annotationCapteur(
    Y_test_pred2.mean() * 28.64 + 11.36, Y_test_pred3.mean() * 1000 + 0, 
    Y_test_pred1.mean() * 1906 + 84, Y_test_pred4.mean() * 45 + 30)

'3.4'